<a href="https://colab.research.google.com/github/chineidu/NLP-Tutorial/blob/main/notebook/06_Transformers/06a_tokenizers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training A New Tokenizer From An Old Tokenizer

- Check [this](https://huggingface.co/learn/nlp-course/chapter6/2?fw=pt) for info on how to finetune a pretrained tokenizer.

In [1]:
!pip install rich
!pip install transformers[torch]
!pip install torch datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00


In [2]:
# Built-in library
import re
import json
from typing import Any, Dict, List, Optional, Union
import logging
import warnings

# Standard imports
import numpy as np
import pandas as pd
from rich import print

# Visualization
import matplotlib.pyplot as plt


# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
# %load_ext lab_black

# auto reload imports
# %load_ext autoreload
# %autoreload 2

<hr><br>

## Batch Encoding Using Fast Tokenizers

In [3]:
from transformers import AutoTokenizer


CHECKPOINT: str = "bert-base-cased"
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
example: str = "My name is Chineidu and I work at Hugging Face In Brooklyn."
encoding: dict["str", Any] = tokenizer(example)

print(type(encoding))

<class 'transformers.tokenization_utils_base.BatchEncoding'>

In [4]:
print(encoding)

# Access the tokens (w/o converting the IDs back to tokens)
print(encoding.tokens())

{
    'input_ids': [
        101,
        1422,
        1271,
        1110,
        19935,
        6851,
        7641,
        1105,
        146,
        1250,
        1120,
        20164,
        10932,
        10289,
        1130,
        6010,
        119,
        102
    ],
    'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

[
    '[CLS]',
    'My',
    'name',
    'is',
    'Chin',
    '##ei',
    '##du',
    'and',
    'I',
    'work',
    'at',
    'Hu',
    '##gging',
    'Face',
    'In',
    'Brooklyn',
    '.',
    '[SEP]'
]

In [5]:
# Get the index of the word each token comes from.
# The special tokens [CLS] and [SEP] are represented as None.
print(encoding.word_ids())

[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

In [6]:
# Try another tokenizer!
CHECKPOINT: str = "roberta-base"
tokenizer_2: AutoTokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)
example_2: str = "My name is Chineidu and I work at Hugging Face In Brooklyn."
encoding_2: dict["str", Any] = tokenizer_2(example_2)

print(encoding_2)

# Access the tokens (w/o converting the IDs back to tokens)
print(encoding_2.tokens())

{
    'input_ids': [0, 2387, 766, 16, 732, 833, 808, 257, 8, 38, 173, 23, 30581, 3923, 12346, 96, 6314, 4, 2],
    'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
}

[
    '<s>',
    'My',
    'Ġname',
    'Ġis',
    'ĠCh',
    'ine',
    'id',
    'u',
    'Ġand',
    'ĠI',
    'Ġwork',
    'Ġat',
    'ĠHug',
    'ging',
    'ĠFace',
    'ĠIn',
    'ĠBrooklyn',
    '.',
    '</s>'
]

In [7]:
print(example)
print(encoding.word_ids())

# Access the tokens (w/o converting the IDs back to tokens)
print(encoding.tokens())

My name is Chineidu and I work at Hugging Face In Brooklyn.

[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, None]

[
    '[CLS]',
    'My',
    'name',
    'is',
    'Chin',
    '##ei',
    '##du',
    'and',
    'I',
    'work',
    'at',
    'Hu',
    '##gging',
    'Face',
    'In',
    'Brooklyn',
    '.',
    '[SEP]'
]

```text
- We can map any word or token to characters in the original text, and vice versa,
* via the:
  - word_to_chars()
  - or token_to_chars() and char_to_word()
  - or char_to_token() methods.
  
- The word_ids() method told us that ##ei is part of the word at index 3, but which word is it in the sentence? We can find out like this:
```

In [8]:
start, end = encoding.word_to_chars(3)
example[start:end]

'Chineidu'

<hr><br>

## [Text Classification Pipeline](https://huggingface.co/learn/nlp-course/chapter6/3?fw=pt)

```text
- Using a token classification pipeline, we can get some results to compare manually.
- The model used by default is dbmdz/bert-large-cased-finetuned-conll03-english and it performs NER on sentences.
```

In [9]:
from transformers import pipeline


TASK: str = "token-classification"  # Named Entity Recognition (NER)
token_classifier: pipeline = pipeline(task=TASK)
example: str = "My name is Chineidu and I work at Hugging Face In Brooklyn."

token_classifier(example)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity': 'I-PER',
  'score': 0.99802446,
  'index': 4,
  'word': 'Chin',
  'start': 11,
  'end': 15},
 {'entity': 'I-PER',
  'score': 0.96976656,
  'index': 5,
  'word': '##ei',
  'start': 15,
  'end': 17},
 {'entity': 'I-PER',
  'score': 0.99290186,
  'index': 6,
  'word': '##du',
  'start': 17,
  'end': 19},
 {'entity': 'I-ORG',
  'score': 0.99207014,
  'index': 11,
  'word': 'Hu',
  'start': 34,
  'end': 36},
 {'entity': 'I-ORG',
  'score': 0.99378514,
  'index': 12,
  'word': '##gging',
  'start': 36,
  'end': 41},
 {'entity': 'I-ORG',
  'score': 0.9924396,
  'index': 13,
  'word': 'Face',
  'start': 42,
  'end': 46},
 {'entity': 'I-LOC',
  'score': 0.9217939,
  'index': 15,
  'word': 'Brooklyn',
  'start': 50,
  'end': 58}]

<br>

#### Comment

```text
- The model properly identified each token generated by `Chineidu` as a person, each token generated by “Hugging Face” as an organization, and the token “Brooklyn” as a location. We can also ask the pipeline to group together the tokens that correspond to the same entity:
```

In [10]:
from transformers import pipeline


TASK: str = "token-classification"  # Named Entity Recognition (NER)

# With "simple" the score is just the mean of the scores of each token in the
# given entity: e.g., the score of “Chineidu” is the mean of the scores
# we saw in the previous example for the tokens Chin, ##ei, and ##du
token_classifier: pipeline = pipeline(task=TASK, aggregation_strategy="simple")
example: str = "My name is Chineidu and I work at Hugging Face In Brooklyn."

token_classifier(example)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'entity_group': 'PER',
  'score': 0.98689765,
  'word': 'Chineidu',
  'start': 11,
  'end': 19},
 {'entity_group': 'ORG',
  'score': 0.99276495,
  'word': 'Hugging Face',
  'start': 34,
  'end': 46},
 {'entity_group': 'LOC',
  'score': 0.9217939,
  'word': 'Brooklyn',
  'start': 50,
  'end': 58}]

#### Other Strategies:

```text
- "first", where the score of each entity is the score of the first token of that entity (so for “Chineidu” it would be 0.99802446, the score of the token Chin)

- "max", where the score of each entity is the maximum score of the tokens in that entity (so for “Hugging Face” it would be 0.98879766, the score of “Face”)

- "average", where the score of each entity is the average of the scores of the words composing that entity (so for “Chineidu” there would be no difference from the "simple" strategy, but “Hugging Face” would have a score of 0.9819, the average of the scores for “Hugging”, 0.975, and “Face”, 0.98879)
```

In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example = "My name is Sylvain and I work at Hugging Face in Brooklyn."
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)

In [11]:
from transformers import AutoTokenizer, AutoModelForTokenClassification


model_checkpoint: str = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model: AutoModelForTokenClassification = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

example: str = "My name is Chineidu and I work at Hugging Face In Brooklyn."
inputs: dict[str, Any] = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [15]:
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [14]:
print(inputs["input_ids"].shape)
print(outputs.logits.shape)

torch.Size([1, 18])

torch.Size([1, 18, 9])

#### Comment:

```text
- The output is a batch with 1 sequence of 18 tokens and the model has 9 different labels, so the output of the model has a shape of 1 x 18 x 9.

- Like for the text classification pipeline, a softmax function is used to convert those logits to probabilities, and the argmax is calculated to get predictions (note that we can take the argmax on the logits because the softmax does not change the order)
```

In [17]:
import torch.nn.functional as F


probabilities: list[float] = F.softmax(outputs.logits, dim=-1)[0].tolist()
predictions: list[int] = outputs.logits.argmax(dim=-1)[0].tolist()
print(predictions)

[0, 0, 0, 0, 4, 4, 4, 0, 0, 0, 0, 6, 6, 6, 0, 8, 0, 0]

In [19]:
# The model.config.id2label attribute contains the mapping of indexes to labels
# that we can use to make sense of the predictions:
print(model.config.id2label)

{0: 'O', 1: 'B-MISC', 2: 'I-MISC', 3: 'B-PER', 4: 'I-PER', 5: 'B-ORG', 6: 'I-ORG', 7: 'B-LOC', 8: 'I-LOC'}

In [23]:
print(probabilities)

[
    [
        0.9991538524627686,
        1.9241502741351724e-05,
        3.707860741997138e-05,
        1.8809221728588454e-05,
        7.031151471892372e-05,
        3.336653026053682e-05,
        0.00020252719696145505,
        2.363589737797156e-05,
        0.00044117661309428513
    ],
    [
        0.9990513920783997,
        1.7267795556108467e-05,
        6.606607348658144e-05,
        1.3313874660525471e-05,
        0.0003467560454737395,
        3.3013118809321895e-05,
        0.0002538103435654193,
        2.5901470507960767e-05,
        0.00019250386685598642
    ],
    [
        0.9993675351142883,
        1.1794357305916492e-05,
        6.0310128901619464e-05,
        1.0065280548587907e-05,
        0.00019843809423036873,
        1.7697746443445794e-05,
        0.00012119061284465715,
        1.845608676376287e-05,
        0.00019445283396635205
    ],
    [
        0.9998182654380798,
        5.2519635573844425e-06,
        1.613173844816629e-05,
        4.3175150494789705e-06,
        4.632448326447047e-05,
        9.044279067893513e-06,
        5.447442890726961e-05,
        9.80641198111698e-06,
        3.649801146821119e-05
    ],
    [
        0.00028339651180431247,
        2.604038490972016e-05,
        7.13504123268649e-05,
        1.6001637050067075e-05,
        0.9980244636535645,
        2.297280116181355e-05,
        0.0003457609855104238,
        2.0992732970626093e-05,
        0.001188966678455472
    ],
    [
        0.011815601028501987,
        0.0004220256523694843,
        0.002812761114910245,
        0.00019349189824424684,
        0.9697666764259338,
        0.00038126061554066837,
        0.006152274552732706,
        0.00021573151752818376,
        0.00824019405990839
    ],
    [
        0.00268651288934052,
        9.38100929488428e-05,
        0.00022670338512398303,
        3.776379526243545e-05,
        0.9929018616676331,
        5.731193959945813e-05,
        0.00247992854565382,
        4.686933971242979e-05,
        0.0014692445984110236
    ],
    [
        0.9998424053192139,
        4.210131464787992e-06,
        1.3797513929603156e-05,
        4.017771971120965e-06,
        2.1001829736633226e-05,
        1.170553241536254e-05,
        6.682379898848012e-05,
        7.474897756765131e-06,
        2.8557964469655417e-05
    ],
    [
        0.9996048808097839,
        1.0593002116365824e-05,
        2.8519587431219406e-05,
        7.79600304667838e-06,
        0.00013510367716662586,
        1.8724107576417737e-05,
        0.00011551325587788597,
        1.626218727324158e-05,
        6.260660302359611e-05
    ],
    [
        0.9997449517250061,
        6.083031621528789e-06,
        1.6914626030484214e-05,
        4.0328850445803255e-06,
        4.9417383706895635e-05,
        1.32603336169268e-05,
        0.00012854274245910347,
        7.682913746975828e-06,
        2.9103030101396143e-05
    ],
    [
        0.999606192111969,
        7.106228622433264e-06,
        2.3774582587066106e-05,
        4.7528851609968115e-06,
        5.372097439249046e-05,
        1.9009909010492265e-05,
        0.00023838413471821696,
        1.0414068128739018e-05,
        3.665811163955368e-05
    ],
    [
        0.005372558254748583,
        2.641263563418761e-05,
        0.00013001101615373045,
        2.4552016839152202e-05,
        0.0017224467592313886,
        9.976174624171108e-05,
        0.9920702576637268,
        3.890434163622558e-05,
        0.0005151150398887694
    ],
    [
        0.003666393458843231,
        6.519862654386088e-05,
        0.0006889182259328663,
        3.467835267656483e-05,
        0.0008976905373856425,
        0.00023536186199635267,
        0.9937851428985596,
        7.348334474954754e-05,
        0.0005531866918317974
    ],
    [
        0.002282232278957963,
        5.6128741562133655e-05,
        0.0007273546070791781,
        5.180847074370831e-05,
        0.002396288327872753,
        0.0002774246677290648,
        0.9924396276473999,
       

#### Note:

```text
- There are 9 labels:
  - O is the label for the tokens that are not in any named entity (it stands for “outside”), and we then have two labels for each type of entity (miscellaneous, person, organization, and location).
  - The label B-XXX indicates the token is at the beginning of an entity XXX and the label I-XXX indicates the token is inside the entity XXX. For instance, in the current example we would expect our model to classify the token `Chin` as B-PER (beginning of a person entity) and the tokens ##ei, and ##du as I-PER (inside a person entity).
```

In [ ]:
# With this map, we are ready to reproduce (almost entirely) the results of the first pipeline
# we can just grab the score and label of each token that was not classified as O:
results: list[str] = []
tokens: list[str] = inputs.tokens()

for idx, pred in enumerate(predictions):
    label = model.config.id2label[pred]
    if label != "O": # label for tokens that's `outside`
        results.append(
            {"entity": label, "score": probabilities[idx][pred], "word": tokens[idx]}
        )

print(results)

In [22]:
model.config.id2label[0]

'O'